In [51]:
import pandas as pd
import numpy as np
from math import pow
import matplotlib as ml
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
ml.style.use('fivethirtyeight')

from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import accuracy_score

In [52]:
data = pd.read_csv('/content/drive/My Drive/dataset.csv')
data.head()

,Marks_1,Marks_2,Result
0,34.623660,78.024693,0
1,30.286711,43.894998,0
2,35.847409,72.902198,0
3,60.182599,86.308552,1
4,79.032736,75.344376,1


In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Marks_1  100 non-null    float64
 1   Marks_2  100 non-null    float64
 2   Result   100 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 2.5 KB


## Logistic regression class

In [54]:
class LogisticRegression():
  def __init__(self,penalty='none',C=0.01,W=0,b=0,epochs=100000,alpha=0.001):
    self.penalty = penalty
    self.C = C
    self.epochs = epochs
    self.W = W
    self.b = b
    self.alpha = alpha
  def sigmoid(self,trainx,weight,bias):
    z = np.dot(trainx,weight) + bias
    return 1/(1 + np.exp(-z))
  def loss(self,h,y):
    if self.penalty == 'none':
      return (-y*np.log(h) - (1-y)*np.log(1-h)).mean()
    elif self.penalty == 'l1':
      return (-y*np.log(h) - (1-y)*np.log(1-h)).mean() + ((1/self.C)*(sum(abs(self.W))))/(2*y.shape[0])
    else:
      return (-y*np.log(h) - (1-y)*np.log(1-h)).mean() + ((1/self.C)*(sum(self.W**2)))/(2*y.shape[0])
  def grad_descent(self,trainx,h,y):
    if self.penalty == 'none':
      return np.dot(trainx.T,(h-y))/y.shape[0],(h-y)/y.shape[0]
    elif self.penalty == 'l1':
      l = (1/self.C)
      return (np.dot(trainx.T,(h-y)) + (l/2))/y.shape[0],(h-y)/y.shape[0]
    else:
      l = (1/self.C)
      return (np.dot(trainx.T,(h-y)) + (l*self.W))/y.shape[0],(h-y)/y.shape[0]
  def update_weights(self,weight,alpha,grad):
    return weight - alpha*grad
  def update_bias(self,bias,alpha,grad_b):
    return bias - alpha*grad_b
  def fit(self,trainx,trainy):
    self.W = np.zeros(trainx.shape[1])
    for i in range(self.epochs):
      h = self.sigmoid(trainx,self.W,self.b)
      grad_desc1,grad_desc2 = self.grad_descent(trainx,h,trainy)
      self.W = self.update_weights(self.W,self.alpha,grad_desc1)
      self.b = self.update_bias(self.b,self.alpha,grad_desc2)
    cost = self.loss(h,trainy)
    return self,self.W,cost
  def predict(self,testx):
    y = self.sigmoid(testx,self.W,self.b)
    for i in range(len(y)):
      if y[i] > 0.5:
        y[i] = 1
      else:
        y[i] = 0
    return y,self.W

In [55]:
# Test
X = np.array(data.iloc[:,:2].values)
y = np.array(data.iloc[:,-1].values)
trainx,testx,trainy,testy = train_test_split(X,y,test_size=0.1,random_state=3)

# L1 REGULARIZATION
log_reg = LogisticRegression(penalty='l1',epochs=100000,alpha=0.001)
_,w,c = log_reg.fit(trainx,trainy)

In [56]:
w

array([ 0.01162761, -0.00229155])

In [57]:
c

0.4527962263117525

In [58]:
# L2 REGULARIZATION
log_reg2 = LogisticRegression(penalty='l2',epochs=100000,alpha=0.001)
_,w2,c2 = log_reg2.fit(trainx,trainy)

In [59]:
w2

array([ 0.01185819, -0.00179029])

In [60]:
c2

0.44477799174247973